In [1]:
import pandas as pd
import networkx as nx 

In [ ]:
def json_to_dataframe(json_data):
    flattened_data = []
    for item in json_data:
        if 'data' in item and 'search_by_raw_query' in item['data'] and 'search_timeline' in item['data']['search_by_raw_query']:
            timeline = item['data']['search_by_raw_query']['search_timeline']
            if 'timeline' in timeline and 'instructions' in timeline['timeline']:
                instructions = timeline['timeline']['instructions']
                for instruction in instructions:
                    if instruction['type'] == 'TimelineAddEntries' and 'entries' in instruction:
                        for entry in instruction['entries']:
                            if 'content' in entry and 'itemContent' in entry['content']:
                                tweet_content = entry['content']['itemContent']
                                if tweet_content['itemType'] == 'TimelineTweet' and 'tweet_results' in tweet_content:
                                    tweet_info = tweet_content['tweet_results']['result']
                                    if 'core' in tweet_info and 'user_results' in tweet_info['core']:
                                        user_info = tweet_info['core']['user_results']['result']

                                        tweet_id = tweet_info.get('rest_id', '')
                                        user_id = user_info.get('rest_id', '')
                                        username = user_info.get('legacy', {}).get('screen_name', '')
                                        description = user_info.get('legacy', {}).get('description', '')
                                        tweet_text = tweet_info.get('legacy', {}).get('full_text', '')
                                        tweet_date = tweet_info.get('legacy', {}).get('created_at', '') 

                                        flattened_data.append({
                                            'tweet_id': tweet_id,
                                            'user_id': user_id,
                                            'username': username,
                                            'description': description,
                                            'tweet_text': tweet_text,
                                            'tweet_date': tweet_date 
                                        })

    return pd.DataFrame(flattened_data)

def combine_json_files_to_dataframe(directory_path):
    all_dataframes = []  
    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):  
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                df = json_to_dataframe(json_data)
                all_dataframes.append(df)

    combined_dataframe = pd.concat(all_dataframes, ignore_index=True)
    return combined_dataframe
